In [ ]:
'''Train a model for the CY-metric'''

In [ ]:
# Generate training data
import numpy as np
import os as os
import pickle as pickle
from cymetric.pointgen.pointgen import PointGenerator

monomials = 5*np.eye(5, dtype=np.int64)
coefficients = np.ones(5)
kmoduli = np.ones(1)
ambient = np.array([4])

pg = PointGenerator(monomials, coefficients, kmoduli, ambient)
points = pg.generate_points(100)
#pg.cy_condition(points)

dirname = 'cy_models/train_data'
n_p = 100000
kappa = pg.prepare_dataset(n_p, dirname)
data = np.load(os.path.join(dirname, 'dataset.npz'))
pg.prepare_basis(dirname, kappa=kappa)


In [ ]:
import tensorflow as tf
tfk = tf.keras
from cymetric.models.callbacks import SigmaCallback
from cymetric.models.tfmodels import PhiFSModel
from cymetric.models.metrics import SigmaLoss
from cymetric.models.tfhelper import prepare_tf_basis, train_model

data = np.load(os.path.join(dirname, 'dataset.npz'))
BASIS = np.load(os.path.join(dirname, 'basis.pickle'), allow_pickle=True)
BASIS = prepare_tf_basis(BASIS)

n_out = 1
nlayer = 3
nHidden = 64
act = 'gelu'
nEpochs = 1
bSizes = [64, 50000]
alpha = [1., 1., 1., 1., 1.]
nfold = 3
n_in = 2*5

nn_phi = tf.keras.Sequential()
nn_phi.add(tfk.Input(shape=(n_in)))
for i in range(nlayer):
    nn_phi.add(tfk.layers.Dense(nHidden, activation=act))
nn_phi.add(tfk.layers.Dense(n_out, use_bias=False))
from cymetric.models.tfmodels import PhiFSModel
phimodel = PhiFSModel(nn_phi, BASIS, alpha=alpha)
opt_phi = tfk.optimizers.Adam()
scb = SigmaCallback((data['X_val'], data['y_val']))
cb_list = [scb]
cmetrics = [SigmaLoss()]

phimodel, training_history = train_model(phimodel, data, optimizer=opt_phi, epochs=nEpochs, batch_sizes=bSizes, 
                                       verbose=1, custom_metrics=cmetrics, callbacks=cb_list)


In [ ]:
#Input testing
print(data.keys())
data_sample = data['X_train'][:2]
print(data_sample.shape, phimodel(data_sample).shape)
print(data_sample)
print(phimodel(data_sample))

In [ ]:
#Save the trained model
save_filepath = os.getcwd()+'/models/cy_models/test_model.keras'
phimodel.save(save_filepath)


In [ ]:
#Import the trained model
imported_model = PhiFSModel(nn_phi, BASIS, alpha=alpha)
imported_model.nn_phi = tfk.models.load_model(save_filepath)
